In [1]:
#libraries
import matplotlib.pyplot as plt
from shapely.geometry import Point
import numpy as np
import folium
import pandas as pd
import json
from folium import plugins
import pytz


#libraries for hotspots specifically
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

In [2]:
#read in the data in this block 
#replace csv read with database call


#set timezone
tz = pytz.timezone('US/Eastern')


### NOTE - please replace csv read below this comment with database call! 
aq = "AQ_orgfid.csv"
data = pd.read_csv(aq, engine='python')
print("Data read!")

#need to drop all rows with lat or long = 0
data = data.loc[(data[['lat', 'long', 'pm25']] != 0).all(axis=1)]

#and grab only pm2.5 values >100 like the R code 
data = data.loc[(data['pm25'] > 100)]

#use the locations of the hotspot clusters to determine where the map displays
#from Canada deployment data code in CityScanner github
def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)

#this allows the code to be used for any deployment instead of being tied to one lat/long location

coords = data.loc[:,['lat','long']].values
start_point=coords[0]

Data read!


In [3]:
#setting up the specifications for the map
bronxMap = folium.Map(location= start_point, tiles='Stamen Terrain', zoom_start=14)

#creating the hotspots on the map
for i,row in data.iterrows():
    folium.CircleMarker((row.lat,row.long), radius=10, weight=2, color='red', fill_color='red', fill_opacity=.5).add_to(bronxMap)

#saving an html version of the map users can zoom in and out of and interact with
bronxMap.save('bronxPointMap.html')

In [4]:
#setting up the specifications for the map
bronxMap = folium.Map(location=start_point, tiles='Stamen Terrain', zoom_start=14)

#creating the hotspots on the map
for i,row in data.iterrows():
    folium.CircleMarker((row.lat,row.long), radius=10, weight=2, color='red', fill_color='red', fill_opacity=.5).add_to(bronxMap)

#adding heatmap functionality
bronxMap.add_children(plugins.HeatMap(data[['lat', 'long']].as_matrix(), radius=25))

#saving an html version of the map users can zoom in and out of and interact with
bronxMap.save('bronxHeatmap.html')

/Users/sanjanapaul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':
/Users/sanjanapaul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.
  if __name__ == '__main__':


In [5]:
centermost_points = clusters.map(get_centermost_point)
start_point=centermost_points[0]

hotspots = data 
coords = hotspots.loc[:,['lat','long']].values

#preprocessing for hotspot clustering. From Canada deployment code in GitHub
kms_per_radian = 6371.0088
epsilon = 0.1 / kms_per_radian
db = DBSCAN(eps=epsilon, min_samples=10, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
cluster_labels = db.labels_
num_clusters = len(set(cluster_labels))-(1 if -1 in set(cluster_labels) else 0)
outliers = coords[cluster_labels == -1]

#creating the clusers. From Canada deployment code in GitHub
clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])
outliers = coords[cluster_labels == -1]
print('Number of clusters: {}'.format(num_clusters))

#setting up the specifications for the map
newmap = folium.Map(location= start_point, tiles='Stamen Toner', zoom_start=14)
points=[]
#add a markers
for index, row in hotspots.iterrows():
    point=(row['lat'], row['long'])
    if point not in points:
        new_point=(row['lat'], row['long'])
        points.append(new_point)      
for rep in centermost_points:
    folium.CircleMarker(location=rep, color='blue', fill=True, fill_color='blue',radius=20).add_to(newmap)
for each in points:
    folium.CircleMarker(location=each, popup='Point:'+str(each), color='red', fill=True, fill_color='red',radius=10).add_to(newmap)
    newmap.add_child(folium.LatLngPopup())
 

NameError: name 'clusters' is not defined

In [ ]:
#interactive html map showing hotspot clusters
newmap.save('bronxHotspotMap.html')